# Import the required packages and libraries

In [1]:
import pickle
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


# Read data

In [2]:
data = pd.read_pickle(r'C:\Users\Waseem\Desktop\Orikami\final_exampleProject.pkl')

# Data analysis

In [3]:
# corr = data.corr()
# corr.style.background_gradient(cmap='coolwarm').set_precision(2)

# Extract features

# plt.hist(data.iloc[:, 0], bins=50)
# plt.show()

# Feature engineering

In [4]:
def extract_features(df, col_names, name, group_size):
    
    winsize = 10
    numwin = round(group_size/winsize)
    start = 0
    ax_mean, ay_mean, az_mean, gx_mean, gy_mean, gz_mean, ax_std, ay_std, az_std, gx_std, gy_std, gz_std = [], [], [], [], [], [], [], [], [], [], [], []
    tdata = []
    labels = []
    for i in range(numwin):
        
        # mean value feature
        ax_mean.append(np.mean(df['acc_x'][start:start+winsize]))
        ay_mean.append(np.mean(df['acc_y'][start:start+winsize]))
        az_mean.append(np.mean(df['acc_z'][start:start+winsize]))
        
        gx_mean.append(np.mean(df['gyr_x'][start:start+winsize]))
        gy_mean.append(np.mean(df['gyr_y'][start:start+winsize]))
        gz_mean.append(np.mean(df['gyr_z'][start:start+winsize]))
        
        # standard deviation value feature
        ax_std.append(np.std(df['acc_x'][start:start+winsize]))
        ay_std.append(np.std(df['acc_y'][start:start+winsize]))
        az_std.append(np.std(df['acc_z'][start:start+winsize]))
        
        gx_std.append(np.std(df['gyr_x'][start:start+winsize]))
        gy_std.append(np.std(df['gyr_y'][start:start+winsize]))
        gz_std.append(np.std(df['gyr_z'][start:start+winsize]))
        labels.append(name)
        
        start = start + winsize
        
   # Create the pandas DataFrame
    tdata.append(ax_mean)
    tdata.append(ay_mean)
    tdata.append(az_mean)
    
    tdata.append(gx_mean)
    tdata.append(gy_mean)
    tdata.append(gz_mean)
    
    tdata.append(ax_std)
    tdata.append(ay_std)
    tdata.append(az_std)
    
    tdata.append(gx_std)
    tdata.append(gy_std)
    tdata.append(gz_std)
    tdata.append(labels)
    tdata = np.transpose(tdata)
    df = pd.DataFrame(tdata, columns=col_names) 
    return df
    

# select data for each category
col_names = ['ax_mean', 'ay_mean', 'az_mean',
            'gx_mean', 'gy_mean', 'gz_mean',
            'ax_std', 'ay_std', 'az_std',
            'gx_std', 'gy_std', 'gz_std', 'gait']

classes = data.groupby(['gait_type'])
newdf = pd.DataFrame(columns=col_names)
class_names = []
for name, group in classes:
    class_names.append(name)
    grp = classes.get_group(str(name))
    df = extract_features(grp, col_names, name, group.shape[0])
    frames = [newdf, df]
    newdf = pd.concat(frames)

# Split data into train, test sets

In [5]:
target = pd.DataFrame(newdf['gait'])
data = newdf.drop(['gait'], axis=1)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(target)
encoded_Y = encoder.transform(target)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

X_train, X_test, y_train, y_test = train_test_split(data, dummy_y, test_size=0.33, random_state=0)

C:\Users\Waseem\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Waseem\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Define, train and evaluate deep learning model

In [6]:
from sklearn.model_selection import StratifiedKFold

seed = 0
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []

# create mode
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(7, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, epochs=33, batch_size=10, verbose=0)
# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
acc: 97.48%
97.48% (+/- 0.00%)


# Model predictions

In [9]:
temp = X_test.iloc[0:1]
y_pred = model.predict_classes(temp)
print(y_pred)
with open(r'C:\Users\Waseem\Desktop\Orikami\xtest.pkl','wb') as f: 
    pickle.dump(temp, f)
# temp.to_pickle(r'C:\Users\Waseem\Desktop\Orikami\xtest.pkl')  # where to save it, usually as a .pkl
# with open(r'C:\Users\Waseem\Desktop\Orikami\xtest.pkl','rb') as f: 
#     xt = pd.DataFrame(pickle.load(f)).to_json()
# print(xt)

[1]


# Save the classification model

In [26]:
import pickle
pickle.dump(model, open(r'C:\Users\Waseem\Desktop\Orikami\gait_predictor.pickle', 'wb'))

# Flask API

In [3]:
from flask import Flask, request, jsonify
import numpy as np
import pickle as p
from keras import backend as K


app = Flask(__name__)


def load_model():

    global model
    model = None
    # model variable refers to the global variable
    with open('gait_predictor.pickle', 'rb') as f:
        model = p.load(f)


@app.route('/')
def home_endpoint():
    return 'Hello World!'


@app.route('/prediction', methods=['GET', 'POST'])
def prediction():

    # jdata = request.get_json()
    # print('Data at Server:', jdata)
    # data = pd.read_json(json.dumps(jdata), orient='records')
    # print("After conversion: ", data)
    load_model()
    with open('xtest.pkl', 'rb') as f:
        data = p.load(f)
    predictions = np.array2string(model.predict_classes(data[0:1]))
    K.clear_session()
    print(jsonify(predictions))
    return jsonify(predictions)

app.run(host='0.0.0.0', port=6000, debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\Waseem\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Request gait predictions

In [ ]:
import requests
import json
import pandas as pd
import pickle
import numpy as np


url = 'http://127.0.0.1:5000/prediction'

with open(r'C:\Users\Waseem\Desktop\Orikami\xtest.pkl', 'rb') as f:
     data = pickle.load(f)
     data = data.to_json(orient='records')
     print(type(data))

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
r = requests.post(url, data=data, headers=headers)
print(r, r.text)